In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import os
from pathlib import Path

In [2]:
SAMPLING_RATE = 12000 # Частота дискретизации 
WINDOW_SIZE = 4096 # Размер фрейма для дробления исходных данных   
STEP = 2048 # Шаг фрейма


NPERSEG = 2048  # Размер фрейма для создания спектрограммы            
NOVERLAP = NPERSEG - 1 #коэф наложения фрейма

DPI = 100 # Количество точек на дюйм для изображения
IMG_HEIGHT = 257  # Высота изображения 
IMG_WIDTH = 3585 # Ширина изображения
CLASS_NAMES = ['defect', 'normal'] # Классы модели, важен порядок (смотри train.ipynb)

In [3]:
def create_spectrogram_from_chunk_normalized(chunk_data, fs):
    """
    Создает нормализованную спектрограмму из фрагмента данных в памяти, не сохраняя на диск.
    Возвращает изображение в виде numpy-массива.
    """
    frequencies, times, Zxx = signal.stft(
        chunk_data, 
        fs=fs, 
        nperseg=NPERSEG, 
        noverlap=NOVERLAP, 
        boundary=None
    )
    Zxx_abs = np.abs(Zxx)
    min_val = np.min(Zxx_abs)
    max_val = np.max(Zxx_abs)
    if max_val > min_val:
        Zxx_normalized = (Zxx_abs - min_val) / (max_val - min_val)
    else:
        Zxx_normalized = np.zeros_like(Zxx_abs)
    height, width = IMG_HEIGHT, IMG_WIDTH # Zxx_normalized.shape
    dpi = DPI
    fig_size = (width / dpi, height / dpi)
    fig = plt.figure(figsize=fig_size, dpi=dpi)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(Zxx_normalized, cmap='gray', origin='lower', aspect='auto')
    fig.canvas.draw()
    buf = fig.canvas.buffer_rgba()
    img_array = np.frombuffer(buf, dtype=np.uint8).reshape(fig.canvas.get_width_height()[::-1] + (4,))
    img_array_rgb = img_array[:, :, :3]
    plt.close(fig)
    return img_array_rgb

In [7]:
def predict_vibration_state(csv_path, model_path):
    """
    Загружает CSV файл, обрабатывает его и делает предсказание с помощью модели.
    """
    print(f"Загрузка модели из файла: {model_path}")
    try:
        model = tf.keras.models.load_model(model_path)
    except Exception as e:
        print(f"Ошибка при загрузке модели: {e}")
        return
    print(f"Чтение и обработка файла: {csv_path}")
    try:
        df = pd.read_csv(csv_path)
        if 'vibration' not in df.columns:
            print("Ошибка: в CSV файле отсутствует колонка 'vibration'.")
            return
        signal_data = df['vibration'].values
    except Exception as e:
        print(f"Ошибка при чтении CSV файла: {e}")
        return
    predictions = []
    for i in range(0, len(signal_data) - WINDOW_SIZE + 1, STEP):
        chunk = signal_data[i : i + WINDOW_SIZE]
        spectrogram_img = create_spectrogram_from_chunk_normalized(chunk, SAMPLING_RATE)
        img_tensor = tf.expand_dims(spectrogram_img, 0)
        prediction = model.predict(img_tensor, verbose=0)
        score = tf.nn.sigmoid(prediction[0][0])
        predicted_class_index = 1 if score > 0.5 else 0
        predictions.append(CLASS_NAMES[predicted_class_index])
    if not predictions:
        print("Не удалось создать ни одного фрагмента для анализа. Файл слишком короткий.")
        return
    print(f"\nАнализ завершен. Всего проанализировано фрагментов: {len(predictions)}")
    defect_count = predictions.count('defect')
    normal_count = predictions.count('normal')
    print(f"  - Фрагментов 'normal': {normal_count}")
    print(f"  - Фрагментов 'defect': {defect_count}")
    if defect_count > 0:
        final_verdict = "DEFECT"
        print("Результат: Defect")
    else:
        final_verdict = "NORMAL"
        print("Результат: Norm")
    return final_verdict

In [8]:
PATH_TO_NEW_CSV = 'csv_cwru_data/defect/B007_0.csv'
PATH_TO_MODEL = 'my_models/vibration_classifier_model_512_NORM_e3.keras'

if not os.path.exists(PATH_TO_NEW_CSV):
    print(f"Файл не найден: {PATH_TO_NEW_CSV}. Укажите правильный путь.")
elif not os.path.exists(PATH_TO_MODEL):
    print(f"Модель не найдена: {PATH_TO_MODEL}. Укажите правильный путь.")
else:
    predict_vibration_state(PATH_TO_NEW_CSV, PATH_TO_MODEL)

Загрузка модели из файла: my_models/vibration_classifier_model_512_NORM_e3.keras
Чтение и обработка файла: csv_cwru_data/defect/B007_0.csv

Анализ завершен. Всего проанализировано фрагментов: 58
  - Фрагментов 'normal': 0
  - Фрагментов 'defect': 58
Результат: Defect
